# Access GitHub via OpenAPI

In this notebook we shall attempt to access GitHub via their OpenAPI interface.

Steps:
1. Fetch GitHub OpenAPI specification
2. Examine the specification
3. Generate a Julia client package using that
4. Examine the generated code
5. Examine the generated docs
6. Use it!

In [ ]:
# Clean up before starting
rm("GitHubClient"; force=true, recursive=true)
# Add required packages
# Pkg.add("OpenAPI")

In [ ]:
using Downloads, OpenAPI

## Fetch the specification

GitHub makes available their OpenAPI specification at:
https://github.com/github/rest-api-description

In [ ]:
spec_file = "specifications/api.github.com.json" 
spec_source = "https://raw.githubusercontent.com/github/rest-api-description/main/descriptions/api.github.com/api.github.com.json"

if !isfile(spec_file)
    Downloads.download(spec_source, spec_file)
end

## Examine the specification

We can examine the API specifications
- Either by opening the JSON specification directly
- Or by opening it in some tool, e.g. the Swagger UI

The Swagger UI tool can be invoked using the OpenAPI package. It uses docker to launch the Swagger tools container, which provides the UI.

In [ ]:
# Examine API specifications
OpenAPI.swagger_ui("specifications/api.github.com.json")

## Code generation

Use openapi-generator to generate the Julia package.
It is available bundled in a convenient docker image,
which lets us use it without having to install anything.

In [ ]:
generator_script = joinpath(pwd(), "generate_github.sh")
run(`$generator_script`);

## Examine the generated code

The code is generated as a Julia module.
The module can be embedded into another package.

Or it can also be registered directly as a Julia package on its own. Just need to add Project.toml.

README.md contains docs for all APIs.

In [ ]:
run(`ls -la GitHubClient`);

In [ ]:
run(`ls -la GitHubClient/src`);

## Use Generated Package to Invoke APIs

The APIs can be invoked just like Julia methods.
All the HTTP/REST communication and serialization and deserialization are handled transparently.

In [ ]:
include("GitHubClient/src/GitHubClient.jl")
using .GitHubClient

In [ ]:
# Create a client for the API group which has the API of interest
client = OpenAPI.Clients.Client(GitHubClient.basepath(GitHubClient.ReposApi));
reposapi = GitHubClient.ReposApi(client);

In [ ]:
# Invoke APIs as regular Julia methods
# API responds with the results converted to a Julia type
# It also returns the raw HTTP response as the second return value, for advanced use
julia_repo, _api_resp = GitHubClient.repos_get(reposapi, "julialang", "julia")

julia_repo

In [ ]:
# The return value is made available as a Julia struct

println("The ", julia_repo.name, " repo has ", julia_repo.stargazers_count, " stars!")

In [ ]:
# Clean up before closing

rm("GitHubClient"; force=true, recursive=true)
OpenAPI.stop_swagger();